In [247]:
import polars as pl
from Levenshtein import ratio

Constants

In [248]:
RATIO = .7 # similarity between T&Cs and gateway list
PARTIAL_RATIO = .5

In [249]:
#1258
terms = (
    pl.scan_csv("data/terms.csv", infer_schema_length=10000) 
)

In [250]:
#1146
gateway = (
    pl.scan_csv('data/gateway_connections.csv', infer_schema_length=10000)
    .with_columns(
        pl.col('Licensee').str.replace_all(r'\([^)]*\)', '').str.strip_chars(' )').str.to_uppercase().alias('licensee_split'),
    )
)


In [251]:
#Should be 112 extra in org_name
combo = (
    terms
    .join(
        gateway, how='left', left_on='org_name', right_on='licensee_split'
    )
    .collect()
    .with_columns(
        pl.col('org_name').map_elements(lambda x: PARTIAL_RATIO if x else RATIO).alias('ratio_check'),
        
        pl.col('Licensee').str.replace_all(r'\([^)]*\)', '').str.strip_chars(' )').str.to_uppercase(),
    )
    .lazy()
   
)


In [252]:
idk = (
    combo
    .with_columns(
        pl.struct(['org_name', 'Licensee']).map_elements(lambda x: ratio(x['org_name'], x['Licensee'])).alias('ratio')
        
    )
     .select(
       'file', 'org_name', 'Licensee', 'ratio'
    )
)

In [253]:
idk.collect().write_excel(
    'data/test.xlsx',
    worksheet='match',
    autofit=True,
    freeze_panes=((1,0,0,0))
)